# Analyzing Used Car Online Listings in Germany
### Cleaning and analyzing data exercise
I will later addd a description to this proyect 

In [1]:
#Importing packages, reading files 
import pandas as pd
import numpy as np 

autos = pd.read_csv('autos.csv', encoding= 'Latin-1') 
#changed default encoding to succesfully read the file 

In [2]:
print(autos.head(),
     autos.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

From the first glance using the `Dataframe.info()` method we can see some interesting thins about our dataset.
1. Columns such as `vehicleType` have `NaN` values
2. Columns have uppercase letters
3. Columns such as `price` are of type `string` (and not `float` or `int`)
    
We will try to clean the data from these inperfections and any other issues that arise    

In [3]:
#Lets begin by checking out the columns and making some changes
print(autos.columns)  

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')


In [4]:
autos_copy = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'ab_test',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen'] #Copied the output above and made some changes

autos.columns = autos_copy
autos.head()

,date_crawled,name,seller,offer_type,price,ab_test,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


## Changes made to the original headers
I have manually changed the original column headers to make the data easier to work with. I changed some columns from `monthOfRegistration` or `notRepairedDamage` to `registration_month` and `unreparied_damage` repectively. I did this to make the headers clearer and easier to understand. In addition to changing some columns names, I changes all of the columns to a snakecase for consistency (previously camelcase). 

*These changes are a good step but lets keep exploring to see what else can we clean in this dataset*

In [5]:
print(autos.describe(include = 'all'))

               date_crawled         name  seller offer_type  price ab_test  \
count                 50000        50000   50000      50000  50000   50000   
unique                48213        38754       2          2   2357       2   
top     2016-04-04 16:40:33  Ford_Fiesta  privat    Angebot     $0    test   
freq                      3           78   49999      49999   1421   25756   
mean                    NaN          NaN     NaN        NaN    NaN     NaN   
std                     NaN          NaN     NaN        NaN    NaN     NaN   
min                     NaN          NaN     NaN        NaN    NaN     NaN   
25%                     NaN          NaN     NaN        NaN    NaN     NaN   
50%                     NaN          NaN     NaN        NaN    NaN     NaN   
75%                     NaN          NaN     NaN        NaN    NaN     NaN   
max                     NaN          NaN     NaN        NaN    NaN     NaN   

       vehicle_type  registration_year  gearbox      power_ps  

We can note above that the `price` and `odometer` columns are stored as text values instead of `float` or `int` values. Another interesting observation is that the `seller` and `offer_type` have almost all the same values except for one data point on each column. 

We will start by tackling the first observation and convert the string solumns to numerical values

In [6]:
#first lets explore how are our dta points saved and note for patterns
print(autos['price'].head(),
     autos['odometer'].value_counts())

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object 150,000km    32424
125,000km     5170
100,000km     2169
90,000km      1757
80,000km      1436
70,000km      1230
60,000km      1164
50,000km      1027
5,000km        967
40,000km       819
30,000km       789
20,000km       784
10,000km       264
Name: odometer, dtype: int64


In [7]:
#Now that we have seen the patterns lets remove non-numeric characters
autos['price'] = (autos['price'].str.replace('$', '')
                 .str.replace(',', '')
                 .astype(int) #change data type
                 )
autos['odometer'] = (autos['odometer'].str.replace(',', '')
                    .str.replace('km', '') 
                     .astype(int) #change data type
                    )

#Now lets change the name of the columns as no to lose the information
autos.rename({'odometer':'odometer_km'}, axis = 1, inplace = True)

#Let check the results
print(autos['price'].dtype,
     autos['odometer_km'].dtype)

int64 int64


In [8]:
#Lets dig deeper on these two columns that are very important for our analysis
print(autos['price'].unique().shape, 
     autos['price'].describe(),
     autos['price'].value_counts().sort_index(ascending = False).head(10)
     )

print(autos['odometer_km'].unique().shape, 
     autos['odometer_km'].describe(),
     autos['odometer_km'].value_counts()
     )

(2357,) count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64 99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
Name: price, dtype: int64
(13,) count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64 150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64


In [9]:
#Lets only keep rows where the price is between 1 and 1 million
autos = autos[autos['price'].between(1, 1000000)]
print(autos['price'].unique().shape,
      autos['price'].describe())
      

(2348,) count     48568.000000
mean       5950.340656
std       11963.134750
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      999999.000000
Name: price, dtype: float64


Now we have cleaned the `price` column from outliers. What we can see from the remaining values is that the std is very large and that mean is far from the median.

In [10]:
#Since the date in the format YYYY-MM-DD is the first ten characters 
#we will extract those characters to work with 
autos['date_crawled'] = autos['date_crawled'].str[:10]
autos['ad_created'] = autos['ad_created'].str[:10]
autos['last_seen'] = autos['last_seen'].str[:10]

#lets explore the frequency per dates in ascending order
print(autos['date_crawled'].value_counts(normalize = True, dropna = False).sort_index(),
      autos['ad_created'].value_counts(normalize = True, dropna = False).sort_index(),
      autos['last_seen'].value_counts(normalize = True, dropna = False).sort_index()
     )
     

2016-03-05    0.025325
2016-03-06    0.014042
2016-03-07    0.036011
2016-03-08    0.033294
2016-03-09    0.033088
2016-03-10    0.032182
2016-03-11    0.032573
2016-03-12    0.036917
2016-03-13    0.015669
2016-03-14    0.036547
2016-03-15    0.034282
2016-03-16    0.029608
2016-03-17    0.031646
2016-03-18    0.012910
2016-03-19    0.034776
2016-03-20    0.037885
2016-03-21    0.037391
2016-03-22    0.032985
2016-03-23    0.032223
2016-03-24    0.029340
2016-03-25    0.031605
2016-03-26    0.032202
2016-03-27    0.031090
2016-03-28    0.034858
2016-03-29    0.034117
2016-03-30    0.033685
2016-03-31    0.031832
2016-04-01    0.033685
2016-04-02    0.035476
2016-04-03    0.038606
2016-04-04    0.036485
2016-04-05    0.013095
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64 2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021

#### When looking at the distribution in the frequency of the dates we can se some notable patterns:

The crawler extracted information from the listings in the course of a month, doing so in a consistent manner.

The listings themselves were created in the span of almost a year. However, most of the listings come from the more recent months such as march 2016.

Lastly, we can clearly note that the the crawler last saw the listings in the last months of its crwaling. (with close to 50% in the last three days)

In [11]:
#Now lets explore the registration year
autos['registration_year'].describe()

count    48568.000000
mean      2004.754612
std         88.641262
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

When cheking the registration years of the used cars for sale we can see that there are some problems with our data:
1. Some data is missing
2. The minimum is 1000 which can't be possible 
3. The maximum is year 9999 which is clearly wrong


In [12]:
#Count the # vehicles with registration after the year 2016 or before year 1920
#(Which we consider wrong and should probably remove)
autos_selection = autos[(autos['registration_year'] > 2016) | (autos['registration_year'] < 1920)]
print(autos_selection.iloc[:,0].count())

1889


In [13]:
#We will remove alls rows that have unrealistic registration_years
autos['registration_year'] = autos.loc[autos['registration_year'].between(1920, 2016),'registration_year']

print(autos['registration_year'].describe(),
      autos['registration_year'].value_counts(normalize=True).sort_index()
     )

count    46679.000000
mean      2002.919985
std          7.121876
min       1927.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64 1927.0    0.000021
1929.0    0.000021
1931.0    0.000021
1934.0    0.000043
1937.0    0.000086
1938.0    0.000021
1939.0    0.000021
1941.0    0.000043
1943.0    0.000021
1948.0    0.000021
1950.0    0.000064
1951.0    0.000043
1952.0    0.000021
1953.0    0.000021
1954.0    0.000043
1955.0    0.000043
1956.0    0.000086
1957.0    0.000043
1958.0    0.000086
1959.0    0.000129
1960.0    0.000493
1961.0    0.000129
1962.0    0.000086
1963.0    0.000171
1964.0    0.000257
1965.0    0.000364
1966.0    0.000471
1967.0    0.000557
1968.0    0.000557
1969.0    0.000407
            ...   
1987.0    0.001542
1988.0    0.002892
1989.0    0.003728
1990.0    0.007434
1991.0    0.007262
1992.0    0.007926
1993.0    0.009105
1994.0    0.013475
1995.0    0.026286
1996.0    0.029414
1997.

Now that we have removed the inexplicable `registration_year` values, we can se that the averaga year is 2003 and that the mayority of the used cars for sale are from the years 1997-2009 (inclusive)

In [14]:
#Explore car brands
print(autos['brand'].value_counts()[:10])

        

volkswagen       10338
opel              5277
bmw               5274
mercedes_benz     4652
audi              4168
ford              3383
renault           2325
peugeot           1430
fiat              1262
seat               920
Name: brand, dtype: int64


We can see that the most common car brands in the used car listings are: volkswagen, opel, bmw, mercedes and audi.

These are the brands that we will aggregate and calculate the average prices

In [15]:
brands = ['volkswagen', 'opel', 'bmw' , 'mercedes_benz', 'audi', 'ford', 'renault', 'peugeot', 'fiat', 'seat']
brands_avg_price = {}

for b in brands:
    mean = autos.loc[autos['brand'] == b,'price'].mean()
    brands_avg_price[b] = mean
    
for key, value in brands_avg_price.items():
    print('{}: {:,.0f}'.format(key, value))

peugeot: 3,066
renault: 2,431
mercedes_benz: 8,536
opel: 2,945
volkswagen: 5,525
ford: 4,023
audi: 9,213
fiat: 2,794
bmw: 8,261
seat: 4,316


Now we can see that the brands such as audi, mercedes benz, and bmw are much more expensive as brands such as fiat or renault. Let calculate the average miles per brand to see if the milage might provide an explaination to this observed pattern. 

In [20]:
brand_avg_milage = {}

for b in brands:
    mean = autos.loc[autos['brand'] == b, 'odometer_km'].mean()
    brand_avg_milage[b] = mean

for key , value in brand_avg_milage.items():
    print('{}: {:,.0f} avg kms per listed car'.format(key, value))

peugeot: 127,357 avg kms per listed car
renault: 128,062 avg kms per listed car
mercedes_benz: 130,796 avg kms per listed car
opel: 129,383 avg kms per listed car
volkswagen: 128,901 avg kms per listed car
ford: 124,350 avg kms per listed car
audi: 129,493 avg kms per listed car
fiat: 117,567 avg kms per listed car
bmw: 132,683 avg kms per listed car
seat: 121,641 avg kms per listed car


In [23]:
#Now lets transform both series objects into one Dataframe for comparison
series_avg_price = pd.Series(brands_avg_price)
df_comparison = pd.DataFrame(series_avg_price, columns=['mean_price'])
df_comparison['mean_kms'] = pd.Series(brand_avg_milage)
print(df_comparison)

                mean_price       mean_kms
audi           9212.930662  129492.562380
bmw            8261.382442  132682.973075
fiat           2793.870048  117567.353407
ford           4022.905409  124349.689625
mercedes_benz  8536.027085  130796.431642
opel           2944.607542  129383.172257
peugeot        3065.611888  127356.643357
renault        2431.195699  128062.365591
seat           4315.744565  121641.304348
volkswagen     5524.906752  128900.657767


When looking at the DataFrame we can see that the most expensive brands are not the brands with the highest avg km per listing. In fact quite the oposite is true. Fiat, for example has the lowest average kms per used car listing and it is has the lowest average listing price. 

This shows us that sellers of premium brands do believe that their car is worth more than the average car even if it has more kilometers than other car brands. (This could be do to the colection cars)